In [ ]:
! pip install goatools

In [17]:
import os
import pandas as pd
import requests

from goatools.obo_parser import GODag
from goatools.godag.go_tasks import get_go2ancestors

In [ ]:
SAVE_PATH = "../../datasets/"

# Ensure the save path exists
os.makedirs(SAVE_PATH, exist_ok=True)

df = pd.read_csv("../../datasets/predictor_dataset.csv")

print("Dataframe shape:", df.shape)

Dataframe shape: (1261, 17)


In [9]:
# Uniprot IDs
df_moondb_dataset = pd.read_csv("../../datasets/moondb_dataset.csv")
df_moonprot_dataset = pd.read_csv("../../datasets/moonprot_dataset.csv")

uniprot_ids_moondb = set(df_moondb_dataset["UniProt IDs"])
uniprot_ids_moonprot = set(df_moonprot_dataset["UniProt IDs"])

In [10]:
# Compare the duplicated rows with the same Unique ID
duplicates = df[df.duplicated(subset=["UniProt IDs"], keep=False)]
print(f"Duplicated rows shape: {duplicates.shape}")

for id in duplicates["UniProt IDs"].unique():
    # Check which dataset the ID belongs to
    if id in uniprot_ids_moondb and id in uniprot_ids_moonprot:
        dataset_info = "in both datasets"
    elif id in uniprot_ids_moondb:
        dataset_info = "in MoonDB dataset"
    elif id in uniprot_ids_moonprot:
        dataset_info = "in MoonProt dataset"
    else:
        dataset_info = "in none of the datasets"

    print(f"Duplicated ID: {id} ({dataset_info})")
    aux = duplicates[duplicates["UniProt IDs"] == id]
    if len(aux) > 1:  # Ensure there are at least two rows to compare
        # Compare each column for the first two rows
        row1, row2 = aux.iloc[0], aux.iloc[1]
        unequal_columns = [col for col in aux.columns if row1[col] != row2[col]]
        print(f"Columns with differences: {', '.join(unequal_columns)}")

Duplicated rows shape: (24, 17)
Duplicated ID: P09169 (in MoonProt dataset)
Columns with differences: Class
Duplicated ID: O75400 (in MoonDB dataset)
Columns with differences: CC_Pairs_Max_MBL, Class
Duplicated ID: P08254 (in MoonProt dataset)
Columns with differences: GO BP Terms, GO CC Terms, MF_Pairs_Max_MBL, Class
Duplicated ID: O75821 (in MoonProt dataset)
Columns with differences: PDB ID, Class
Duplicated ID: O15371 (in MoonProt dataset)
Columns with differences: PDB ID, MF_Pairs_Max_MBL, Class
Duplicated ID: P10096 (in MoonProt dataset)
Columns with differences: Class
Duplicated ID: O14788 (in MoonDB dataset)
Columns with differences: GO BP Terms, Max_MBL_MF, MF_Pairs_Max_MBL, Highest_MBL, Class
Duplicated ID: P00004 (in MoonProt dataset)
Columns with differences: Class
Duplicated ID: P15822 (in MoonDB dataset)
Columns with differences: CC_Pairs_Max_MBL, Class
Duplicated ID: O43639 (in MoonDB dataset)
Columns with differences: CC_Pairs_Max_MBL, Class
Duplicated ID: P12063 (in bo

In [12]:
# Drop both copies of the duplicated rows based on UniProt IDs
df = df[~df["UniProt IDs"].isin(duplicates["UniProt IDs"])]
print("Dataframe shape after dropping all duplicates:", df.shape)

Dataframe shape after dropping all duplicates: (1237, 17)


In [13]:
# Print the na values of each column
print("NA values in each column:")
print(df.head())
print(df.isna().sum())

NA values in each column:
  UniProt IDs                                             PDB ID  \
1      Q9Y2X8                                                NaN   
2      Q05086  1C4Z; 1D5F; 1EQX; 2KR1; 4GIZ; 4XR8; 6SJV; 6SLM...   
3      Q9Y6X0                                                NaN   
4      Q8BH75                                               2OGB   
5      Q04120                                   5DVB; 5EPT; 6UTL   

                                           Gene Name  \
1         {'Name': 'UBE2D4', 'Synonyms': ['UBCH5D']}   
2  {'Name': 'UBE3A {ECO:0000312|HGNC:HGNC:12496}'...   
3       {'Name': 'SETBP1', 'Synonyms': ['KIAA0437']}   
4   {'Name': 'Rnf41', 'Synonyms': ['Flrf', 'Nrdp1']}   
5  {'Name': 'TSA2 {ECO:0000303|PubMed:11741925}',...   

                         Protein Name  \
1  Ubiquitin-conjugating enzyme E2 D4   
2        Ubiquitin-protein ligase E3A   
3                 SET-binding protein   
4   E3 ubiquitin-protein ligase NRDP1   
5    Peroxiredoxin TSA2

In [16]:
# Define the file path and URL for the GO DAG file
obo_file_path = "go-basic.obo"
obo_url = "http://current.geneontology.org/ontology/go-basic.obo"

# Check if the file exists, otherwise download it
if not os.path.exists(obo_file_path):
    print(f"{obo_file_path} not found. Downloading...")
    response = requests.get(obo_url)
    if response.status_code == 200:
        with open(obo_file_path, "wb") as f:
            f.write(response.content)
        print(f"Downloaded go-basic.obo to {obo_file_path}")
    else:
        raise ValueError(f"Failed to download go-basic.obo. HTTP status code: {response.status_code}")

# Load the GO DAG
print("Loading GO DAG...")
godag = GODag(obo_file_path)
print(f"Number of GO terms loaded: {len(godag)}")

go-basic.obo not found. Downloading...
Downloaded go-basic.obo to go-basic.obo
Loading GO DAG...
go-basic.obo: fmt(1.2) rel(2025-03-16) 43,544 Terms
Number of GO terms loaded: 43544


In [22]:
# Functions kept from the FranciscoJPuiMot repository: https://github.com/CBBIO/protein-metamorphisms-is/blob/main/protein_metamorphisms_is/operation/functional/multifunctionality/go_multifunctionality_metrics.py

def get_all_ancestors(go_id, go2ancestors):
    """Returns all ancestors of the GO term, including optional relationships."""
    return go2ancestors.get(go_id, set())


def calculate_mbl_with_relationships(go_id1, go_id2, godag):
    # Create the subgraph with all relationships
    go2ancestors = get_go2ancestors(set(godag.values()), relationships={"is_a", "part_of", "regulates"})

    # Get all ancestors of both terms
    ancestors1 = get_all_ancestors(go_id1, go2ancestors) | {go_id1}
    ancestors2 = get_all_ancestors(go_id2, go2ancestors) | {go_id2}

    # Find the common ancestors
    common_ancestors = ancestors1.intersection(ancestors2)
    if not common_ancestors:
        print("There are no common ancestors between the given terms.")
        return None

    # Calculate the minimum distance to the common ancestors
    min_distance = float("inf")
    for ancestor in common_ancestors:
        distance = abs(godag[go_id1].depth + godag[go_id2].depth - 2 * godag[ancestor].depth)
        min_distance = min(min_distance, distance)

    print(f"Minimum Branch Length (MBL) between {go_id1} and {go_id2}: {min_distance}")
    return min_distance

def calculate_all_mbl_from_GO_attribute(df, go_attribute):
    """
    Calculates the Minimum Branch Length (MBL) between all pairs of GO terms in the given attribute.
    Returns a list of all MBL values and the maximum MBL for each row (Pairs_Max_MBL).
    """
    mbl_values = []
    pairs_max_mbl = []

    for i in range(len(df)):
        go_id1 = df.iloc[i][go_attribute]
        row_mbl_values = []  # Store MBL values for the current row
        for j in range(i + 1, len(df)):
            go_id2 = df.iloc[j][go_attribute]
            if pd.notna(go_id1) and pd.notna(go_id2):
                mbl = calculate_mbl_with_relationships(go_id1, go_id2, godag)
                if mbl is not None:
                    mbl_values.append(mbl)
                    row_mbl_values.append(mbl)

        # Store the maximum MBL for the current row
        if row_mbl_values:
            pairs_max_mbl.append(max(row_mbl_values))
        else:
            pairs_max_mbl.append(None)  # No valid pairs for this row

    return mbl_values, pairs_max_mbl